## Oxford-IIIT Pet Dataset 을 이용하여 localization을 위한 tfrecord 파일 만들기

이 실습에서는 Oxford-IIIT Pet Dataset을 training과 validation data로 나누고, tfrecord 파일로 저장하는 부분까지 진행합니다.



In [1]:
## library import
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import shutil
import xml.etree.ElementTree as et
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.0.0-beta1


In [2]:
cd '/content/gdrive/My Drive/TensorFlow_Training_13th'

/content/gdrive/My Drive/TensorFlow_Training_13th


In [0]:
## directory 정보
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'oxford_pet')
image_dir = os.path.join(data_dir, 'images')
anno_dir = os.path.join(data_dir, 'annotations', 'xmls')

In [5]:
## localization을 위한 annotation이 되어 있는 file의 수 확인
anno_files = [fname for fname in os.listdir(anno_dir) if os.path.splitext(fname)[-1] == '.xml']
len(anno_files)

3686

In [6]:
## image file의 수 확인
img_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
len(img_files)

7390

In [0]:
## 개와 고양이 directory로 생성
dog_dir = os.path.join(data_dir, 'dogs')
cat_dir = os.path.join(data_dir, 'cats')
os.makedirs(dog_dir, exist_ok=True)
os.makedirs(cat_dir, exist_ok=True)

In [0]:
## segmentation을 위한 annotation이 되어 있는 directory 경로 설정
seg_dir = os.path.join(data_dir, 'annotations', 'trimaps')

In [0]:
'''localization을 위한 annotation이 되어 있는 directory에서 file을 하나씩 읽어서, 
관련된 image file을 찾은 뒤, 개 image는 dogs 디렉토리에, 고양이 image는 cats 디렉토리에 복사
이 때 file 이름이 대문자로 시작하면 고양이, 소문자로 시작하면 개의 image 임을 이용함
또 이 image들이 segmentaion을 위한 annotation도 되어 있는지 확인함'''

cat_cnt = 0
dog_cnt = 0
for anno_file in anno_files:
    ## annotation 파일이름에서 확장자 뺀 앞부분만 추출
    fname = os.path.splitext(anno_file)[0]
    ## segmentation annotation은 png file format
    sname = fname + '.png'
    ## image file은 jpg format
    fname = fname + '.jpg'
    fpath = os.path.join(image_dir, fname)
    ## file 이름의 첫글자가 소문자인지 확인
    if fname[0].islower():
        dog_cnt += 1
        cpath = os.path.join(dog_dir, fname)
    else:
        cat_cnt += 1
        cpath = os.path.join(cat_dir, fname)    
    ## segmentation annotation도 있는지 확인
    if sname not in os.listdir(seg_dir):
        print(sname)
    ## data copy
    shutil.copy(fpath, cpath)

In [10]:
print("총 복사된 image 수: {}".format(dog_cnt+cat_cnt))
print("개 image 수: {}".format(dog_cnt))
print("고양이 image 수: {}".format(cat_cnt))

총 복사된 image 수: 3686
개 image 수: 2498
고양이 image 수: 1188


In [0]:
## validation data를 나누기 위한 directory 생성
val_dir = os.path.join(data_dir, 'dogs_val')
os.makedirs(val_dir, exist_ok=True)

In [0]:
## train data와 validation data를 나누기 위한 index 생성
train_idx, val_idx = train_test_split(np.arange(dog_cnt), train_size=0.80065)

In [14]:
## train, validation data 수 확인
print("train data의 수: {}".format(train_idx.shape[0]))
print("validation data의 수: {}".format(val_idx.shape[0]))

train data의 수: 2000
validation data의 수: 498


In [15]:
## validation data를 validatino directory로 이동
dog_files = os.listdir(dog_dir)

val_cnt = 0
for idx, dog_file in enumerate(dog_files):
    fpath = os.path.join(dog_dir, dog_file)
    mpath = os.path.join(val_dir, dog_file)
    if idx in val_idx:
        shutil.move(fpath, mpath)
        val_cnt += 1
print(val_cnt)
print(dog_cnt - val_cnt)

498
2000


### TFRecord 파일 생성

In [18]:
## library import
import tensorflow as tf
import numpy as np
import os
from PIL import Image

print(tf.__version__)

2.0.0-beta1


In [19]:
cd '/content/gdrive/My Drive/TensorFlow_Training_13th'

/content/gdrive/My Drive/TensorFlow_Training_13th


In [0]:
cur_dir = os.getcwd()
train_dir = os.path.join(data_dir, 'dogs')
val_dir = os.path.join(data_dir, 'dogs_val')

In [0]:
## tfrecord 파일 저장할 directory 생성
tfr_dir = os.path.join(cur_dir, 'dogs_tfr')
os.makedirs(tfr_dir, exist_ok=True)

In [0]:
## tfrecord 파일 경로 설정
tfr_train = 'dogs_train.tfrecord'
tfr_val = 'dogs_val.tfrecord'

tfr_train_dir = os.path.join(tfr_dir, tfr_train)
tfr_val_dir = os.path.join(tfr_dir, tfr_val)

In [0]:
## tfrecord writer
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [0]:
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [25]:
## train data의 tfrecord file 생성
img_size = 224
fnames = os.listdir(train_dir)
n_train = 0
for fname in fnames:
    ## train data directory에서 file을 읽어서 serialize
    fpath = os.path.join(train_dir, fname)
    img = Image.open(fpath)
    img = img.resize((img_size, img_size))
    img = np.asarray(img)
    img_str = img.tobytes()
    
    '''bounding box의 정보가 있는 xml file을 열어서 좌표 정보를 parsing한 후,
    boudnign box의 중심점의 x, y 좌표와 width, height를 계산함
    이 때 0~1사이의 값이 되도록 width와 height로 나눠줌'''
    aname = os.path.splitext(fname)[0] + '.xml'
    apath = os.path.join(anno_dir, aname)
    tree = et.parse(apath)
    ## xml file parsing
    width = float(tree.find('./size/width').text)
    height = float(tree.find('./size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    ## x, y, w, h 계산
    xc = (xmin+xmax)/2.
    yc = (ymin+ymax)/2.
    x = xc/width
    y = yc/height
    w = (xmax-xmin)/width
    h = (ymax-ymin)/height
    
    ## tfrecord file로 write
    example = tf.train.Example(features=tf.train.Features(feature={
            'image': _bytes_feature(img_str),
            'x': _float_feature(x),
            'y': _float_feature(y),
            'w': _float_feature(w),
            'h': _float_feature(h)}))
    writer_train.write(example.SerializeToString())
    n_train += 1

print(n_train)

2000


In [26]:
## train data의 tfrecord file 생성
img_size = 224
fnames = os.listdir(val_dir)
n_val = 0
for fname in fnames:
    ## validation data directory에서 file을 읽어서 serialize
    fpath = os.path.join(val_dir, fname)
    img = Image.open(fpath)
    img = img.resize((img_size, img_size))
    img = np.asarray(img)
    img_str = img.tobytes()
    
    '''bounding box의 정보가 있는 xml file을 열어서 좌표 정보를 parsing한 후,
    boudnign box의 중심점의 x, y 좌표와 width, height를 계산함
    이 때 0~1사이의 값이 되도록 width와 height로 나눠줌'''
    aname = os.path.splitext(fname)[0] + '.xml'
    apath = os.path.join(anno_dir, aname)
    tree = et.parse(apath)
    ## xml file parsing
    width = float(tree.find('./size/width').text)
    height = float(tree.find('./size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    ## x, y, w, h 계산
    xc = (xmin+xmax)/2.
    yc = (ymin+ymax)/2.
    x = xc/width
    y = yc/height
    w = (xmax-xmin)/width
    h = (ymax-ymin)/height
    
    ## tfrecord file로 write
    example = tf.train.Example(features=tf.train.Features(feature={
            'image': _bytes_feature(img_str),
            'x': _float_feature(x),
            'y': _float_feature(y),
            'w': _float_feature(w),
            'h': _float_feature(h)}))
    writer_val.write(example.SerializeToString())
    n_val += 1

print(n_val)

498


In [27]:
## file이 제대로 생성되었는지 확인
!ls -l 'dogs_tfr'

total 367458
-rw------- 1 root root 301262000 Aug  3 00:12 dogs_train.tfrecord
-rw------- 1 root root  75014238 Aug  3 00:12 dogs_val.tfrecord
